# Low-frequency component analysis of SST

Reading in SST data and performing low-frequency component analysis

In [2]:
%matplotlib inline

import xarray as xr
import numpy as np
import intake

# util.py is in the local directory
# it contains code that is common across project notebooks
# or routines that are too extensive and might otherwise clutter
# the notebook design
import util 

## Using `intake-esm` to get SST data
[Intake-esm](https://intake-esm.readthedocs.io) is a data cataloging utility that facilitates access to CMIP data. It's pretty awesome.

An `intake-esm` collection object establishes a link to a database that contains file locations and associated metadata (i.e. which experiement, model, etc. thet come from). 

### Opening a collection
First step is to open a collection by pointing to the collection definition file, which is a JSON file that conforms to the [ESM Collection Specification](https://github.com/NCAR/esm-collection-spec). 

The collection JSON files are stored locally in this repository for purposes of reproducibility---and because Cheyenne compute nodes don't have Internet access. 

The primary source for these files is the [intake-esm-datastore](https://github.com/NCAR/intake-esm-datastore) repository. Any changes made to these files should be pulled from that repo. For instance, the Pangeo cloud collection is available [here](https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json).

In [3]:
if util.is_ncar_host():
    col = intake.open_esm_datastore("../catalogs/glade-cmip6.json")
else:
    col = intake.open_esm_datastore("../catalogs/pangeo-cmip6.json")
col

glade-cmip6-ESM Collection with 698724 entries:
	> 13 activity_id(s)

	> 24 institution_id(s)

	> 47 source_id(s)

	> 68 experiment_id(s)

	> 162 member_id(s)

	> 35 table_id(s)

	> 1027 variable_id(s)

	> 12 grid_label(s)

	> 59 dcpp_init_year(s)

	> 248 version(s)

	> 6813 time_range(s)

	> 698724 path(s)

`intake-esm` is build on top of [pandas](https://pandas.pydata.org/pandas-docs/stable). It is possible to view the `pandas.DataFrame` as follows.

In [4]:
col.df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,path
0,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,day,pr,gn,NaN,v20190702,20150101-20551231,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
1,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,hfls,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
2,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,prsn,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
3,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,va,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
4,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,tas,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...


It is possible to interact with the `DataFrame`; for instance, we can see what the "attributes" of the datasets are by printing the columns.

In [5]:
col.df.columns

Index(['activity_id', 'institution_id', 'source_id', 'experiment_id',
       'member_id', 'table_id', 'variable_id', 'grid_label', 'dcpp_init_year',
       'version', 'time_range', 'path'],
      dtype='object')

### Search and discovery

#### Finding unique entries
Let's query the data to see what models ("source_id"), experiments ("experiment_id") and temporal frequencies ("table_id") are available.

In [6]:
import pprint 
uni_dict = col.unique(['source_id', 'experiment_id', 'table_id'])
pprint.pprint(uni_dict, compact=True)

{'experiment_id': {'count': 68,
                   'values': ['ssp370', 'histSST-piNTCF', 'histSST',
                              'histSST-1950HC', 'hist-1950HC', 'hist-piNTCF',
                              'piClim-NTCF', 'ssp370SST-lowNTCF',
                              'ssp370-lowNTCF', 'ssp370SST', '1pctCO2-bgc',
                              'hist-bgc', 'esm-ssp585', 'amip-future4K',
                              'amip-m4K', 'a4SST', 'aqua-p4K', 'piSST',
                              'amip-4xCO2', 'a4SSTice', 'amip-p4K',
                              'aqua-control', 'aqua-4xCO2', 'abrupt-4xCO2',
                              'historical', 'piControl', 'amip', '1pctCO2',
                              'esm-hist', 'esm-piControl', 'ssp245', 'ssp585',
                              'ssp126', 'hist-GHG', 'hist-aer',
                              'dcppA-hindcast', 'dcppC-hindcast-noPinatubo',
                              'dcppC-hindcast-noElChichon', 'dcppA-assim',
                   

#### Searching for specific datasets

Let's find all the dissolved oxygen data at annual frequency from the ocean for the `historical` and `ssp585` experiments.

In [7]:
cat = col.search(experiment_id=['historical', 'ssp585'], table_id='Amon', variable_id='ts', grid_label='gn')
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,path
33175,CMIP,BCC,BCC-ESM1,historical,r2i1p1f1,Amon,ts,gn,NaN,v20181214,185001-201412,/glade/collections/cmip/CMIP6/CMIP/BCC/BCC-ESM...
33533,CMIP,BCC,BCC-ESM1,historical,r1i1p1f1,Amon,ts,gn,NaN,v20181214,185001-201412,/glade/collections/cmip/CMIP6/CMIP/BCC/BCC-ESM...
33892,CMIP,BCC,BCC-ESM1,historical,r3i1p1f1,Amon,ts,gn,NaN,v20181214,185001-201412,/glade/collections/cmip/CMIP6/CMIP/BCC/BCC-ESM...
35842,CMIP,BCC,BCC-CSM2-MR,historical,r2i1p1f1,Amon,ts,gn,NaN,v20181115,185001-201412,/glade/collections/cmip/CMIP6/CMIP/BCC/BCC-CSM...
36283,CMIP,BCC,BCC-CSM2-MR,historical,r1i1p1f1,Amon,ts,gn,NaN,v20181126,185001-201412,/glade/collections/cmip/CMIP6/CMIP/BCC/BCC-CSM...
...,...,...,...,...,...,...,...,...,...,...,...,...
697374,ScenarioMIP,MRI,MRI-ESM2-0,ssp585,r1i1p1f1,Amon,ts,gn,NaN,v20190222,201501-210012,/glade/collections/cmip/CMIP6/ScenarioMIP/MRI/...
697538,ScenarioMIP,NUIST,NESM3,ssp585,r2i1p1f1,Amon,ts,gn,NaN,v20190731,201501-210012,/glade/collections/cmip/CMIP6/ScenarioMIP/NUIS...
697563,ScenarioMIP,NUIST,NESM3,ssp585,r1i1p1f1,Amon,ts,gn,NaN,v20190728,201501-210012,/glade/collections/cmip/CMIP6/ScenarioMIP/NUIS...
697670,ScenarioMIP,MIROC,MIROC-ES2L,ssp585,r1i1p1f2,Amon,ts,gn,NaN,v20190823,201501-210012,/glade/collections/cmip/CMIP6/ScenarioMIP/MIRO...


It might be desirable to get more specific. For instance, we may want to select only the models that have *both* `historical` and `ssp585` data. We coud do this as follows.

In [8]:
models = set(uni_dict['source_id']['values']) # all the models

for experiment_id in ['historical', 'ssp585']:
    query = dict(experiment_id=experiment_id, table_id='Amon', 
                 variable_id='ts', grid_label='gn')  
    cat = col.search(**query)
    models = models.intersection({model for model in cat.df.source_id.unique().tolist()})

models = list(models)
models

['CAMS-CSM1-0',
 'MIROC-ES2L',
 'NESM3',
 'BCC-CSM2-MR',
 'UKESM1-0-LL',
 'FGOALS-g3',
 'MIROC6',
 'MRI-ESM2-0',
 'CanESM5']

In [9]:
cat = col.search(experiment_id=['historical', 'ssp585'], table_id='Amon', 
                 variable_id='ts', grid_label='gn', source_id=models)
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,path
35842,CMIP,BCC,BCC-CSM2-MR,historical,r2i1p1f1,Amon,ts,gn,NaN,v20181115,185001-201412,/glade/collections/cmip/CMIP6/CMIP/BCC/BCC-CSM...
36283,CMIP,BCC,BCC-CSM2-MR,historical,r1i1p1f1,Amon,ts,gn,NaN,v20181126,185001-201412,/glade/collections/cmip/CMIP6/CMIP/BCC/BCC-CSM...
36720,CMIP,BCC,BCC-CSM2-MR,historical,r3i1p1f1,Amon,ts,gn,NaN,v20181119,185001-201412,/glade/collections/cmip/CMIP6/CMIP/BCC/BCC-CSM...
174939,CMIP,CAS,FGOALS-g3,historical,r2i1p1f1,Amon,ts,gn,NaN,v20190828,195001-195912,/glade/collections/cmip/CMIP6/CMIP/CAS/FGOALS-...
174940,CMIP,CAS,FGOALS-g3,historical,r2i1p1f1,Amon,ts,gn,NaN,v20190828,196001-196912,/glade/collections/cmip/CMIP6/CMIP/CAS/FGOALS-...
...,...,...,...,...,...,...,...,...,...,...,...,...
697374,ScenarioMIP,MRI,MRI-ESM2-0,ssp585,r1i1p1f1,Amon,ts,gn,NaN,v20190222,201501-210012,/glade/collections/cmip/CMIP6/ScenarioMIP/MRI/...
697538,ScenarioMIP,NUIST,NESM3,ssp585,r2i1p1f1,Amon,ts,gn,NaN,v20190731,201501-210012,/glade/collections/cmip/CMIP6/ScenarioMIP/NUIS...
697563,ScenarioMIP,NUIST,NESM3,ssp585,r1i1p1f1,Amon,ts,gn,NaN,v20190728,201501-210012,/glade/collections/cmip/CMIP6/ScenarioMIP/NUIS...
697670,ScenarioMIP,MIROC,MIROC-ES2L,ssp585,r1i1p1f2,Amon,ts,gn,NaN,v20190823,201501-210012,/glade/collections/cmip/CMIP6/ScenarioMIP/MIRO...


### Loading data

`intake-esm` enables loading data directly into an [xarray.Dataset](http://xarray.pydata.org/en/stable/api.html#dataset).

Note that data on the cloud are in 
[zarr](https://zarr.readthedocs.io/en/stable/) format and data on 
[glade](https://www2.cisl.ucar.edu/resources/storage-and-file-systems/glade-file-spaces) are stored as 
[netCDF](https://www.unidata.ucar.edu/software/netcdf/) files. This is opaque to the user.

`intake-esm` has rules for aggegating datasets; these rules are defined in the collection-specification file.

In [10]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': False}, 
                                cdf_kwargs={'chunks': {}, 'decode_times': False})


xarray will load netCDF datasets with dask using a single chunk for all arrays.
For effective chunking, please provide chunks in cdf_kwargs.
For example: cdf_kwargs={'chunks': {'time': 36}}

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 18 group(s)


`dset_dict` is a dictionary of `xarray.Dataset`'s; its keys are constructed to refer to compatible groups.

In [11]:
dset_dict.keys()

dict_keys(['CMIP.BCC.BCC-CSM2-MR.historical.Amon.gn', 'CMIP.CAMS.CAMS-CSM1-0.historical.Amon.gn', 'CMIP.CAS.FGOALS-g3.historical.Amon.gn', 'CMIP.CAS.FGOALS-g3.ssp585.Amon.gn', 'CMIP.CCCma.CanESM5.historical.Amon.gn', 'CMIP.MIROC.MIROC-ES2L.historical.Amon.gn', 'CMIP.MIROC.MIROC6.historical.Amon.gn', 'CMIP.MOHC.UKESM1-0-LL.historical.Amon.gn', 'CMIP.MRI.MRI-ESM2-0.historical.Amon.gn', 'CMIP.NUIST.NESM3.historical.Amon.gn', 'ScenarioMIP.BCC.BCC-CSM2-MR.ssp585.Amon.gn', 'ScenarioMIP.CAMS.CAMS-CSM1-0.ssp585.Amon.gn', 'ScenarioMIP.CCCma.CanESM5.ssp585.Amon.gn', 'ScenarioMIP.MIROC.MIROC-ES2L.ssp585.Amon.gn', 'ScenarioMIP.MIROC.MIROC6.ssp585.Amon.gn', 'ScenarioMIP.MOHC.UKESM1-0-LL.ssp585.Amon.gn', 'ScenarioMIP.MRI.MRI-ESM2-0.ssp585.Amon.gn', 'ScenarioMIP.NUIST.NESM3.ssp585.Amon.gn'])

We can access a particular dataset as follows.

In [23]:
# choose model here
ds_hist = dset_dict['CMIP.CCCma.CanESM5.historical.Amon.gn']
ds_rcp85 = dset_dict['ScenarioMIP.CCCma.CanESM5.ssp585.Amon.gn']

### Get land fraction

As motivation for diving into more advanced manipulations with `intake-esm`, let's consider the task of getting access to grid information in the `Ofx` table_id.

In [14]:
cat_fx = col.search(experiment_id=['historical'], source_id=models, variable_id='sftlf', table_id='fx', grid_label='gn')
cat_fx.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,path
507443,CMIP,MRI,MRI-ESM2-0,historical,r2i1p1f1,fx,sftlf,gn,NaN,v20190603,NaN,/glade/collections/cmip/CMIP6/CMIP/MRI/MRI-ESM...
507710,CMIP,MRI,MRI-ESM2-0,historical,r5i1p1f1,fx,sftlf,gn,NaN,v20190603,NaN,/glade/collections/cmip/CMIP6/CMIP/MRI/MRI-ESM...
508004,CMIP,MRI,MRI-ESM2-0,historical,r1i1p1f1,fx,sftlf,gn,NaN,v20190603,NaN,/glade/collections/cmip/CMIP6/CMIP/MRI/MRI-ESM...
508273,CMIP,MRI,MRI-ESM2-0,historical,r4i1p1f1,fx,sftlf,gn,NaN,v20190603,NaN,/glade/collections/cmip/CMIP6/CMIP/MRI/MRI-ESM...
508549,CMIP,MRI,MRI-ESM2-0,historical,r3i1p1f1,fx,sftlf,gn,NaN,v20190603,NaN,/glade/collections/cmip/CMIP6/CMIP/MRI/MRI-ESM...
514454,CMIP,MIROC,MIROC6,historical,r2i1p1f1,fx,sftlf,gn,NaN,v20190311,NaN,/glade/collections/cmip/CMIP6/CMIP/MIROC/MIROC...
515235,CMIP,MIROC,MIROC6,historical,r5i1p1f1,fx,sftlf,gn,NaN,v20190311,NaN,/glade/collections/cmip/CMIP6/CMIP/MIROC/MIROC...
516016,CMIP,MIROC,MIROC6,historical,r1i1p1f1,fx,sftlf,gn,NaN,v20190311,NaN,/glade/collections/cmip/CMIP6/CMIP/MIROC/MIROC...
516797,CMIP,MIROC,MIROC6,historical,r4i1p1f1,fx,sftlf,gn,NaN,v20190311,NaN,/glade/collections/cmip/CMIP6/CMIP/MIROC/MIROC...
517578,CMIP,MIROC,MIROC6,historical,r3i1p1f1,fx,sftlf,gn,NaN,v20190311,NaN,/glade/collections/cmip/CMIP6/CMIP/MIROC/MIROC...


In [15]:
df = cat_fx.df.copy()
df.drop_duplicates(subset=['source_id', 'variable_id'], inplace=True)
df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,path
507443,CMIP,MRI,MRI-ESM2-0,historical,r2i1p1f1,fx,sftlf,gn,NaN,v20190603,NaN,/glade/collections/cmip/CMIP6/CMIP/MRI/MRI-ESM...
514454,CMIP,MIROC,MIROC6,historical,r2i1p1f1,fx,sftlf,gn,NaN,v20190311,NaN,/glade/collections/cmip/CMIP6/CMIP/MIROC/MIROC...


Now, since we've only retained one ensemble member, we need to eliminate that column. If we omit this step, `intake-esm` will throw an error, complaining that different variables are present for each ensemble member. Setting the `member_id` column to NaN precludes attempts to join along the ensemble dimension.

After this final manipulation, we copy the `DataFrame` back to the collection object and proceed with loading the data.

In [16]:
df['member_id'] = np.nan
cat_fx.df = df

In [17]:
fx_dsets = cat_fx.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})


xarray will load netCDF datasets with dask using a single chunk for all arrays.
For effective chunking, please provide chunks in cdf_kwargs.
For example: cdf_kwargs={'chunks': {'time': 36}}

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 2 group(s)


In [18]:
fx_dsets.keys()

dict_keys(['CMIP.MIROC.MIROC6.historical.fx.gn', 'CMIP.MRI.MRI-ESM2-0.historical.fx.gn'])

In [19]:
for key, ds in fx_dsets.items():
    print(ds.data_vars)

Data variables:
    lat_bnds  (lat, bnds) float64 dask.array<chunksize=(128, 2), meta=np.ndarray>
    lon_bnds  (lon, bnds) float64 dask.array<chunksize=(256, 2), meta=np.ndarray>
    type      |S4 ...
    sftlf     (lat, lon) float32 dask.array<chunksize=(128, 256), meta=np.ndarray>
Data variables:
    lat_bnds  (lat, bnds) float64 dask.array<chunksize=(160, 2), meta=np.ndarray>
    lon_bnds  (lon, bnds) float64 dask.array<chunksize=(320, 2), meta=np.ndarray>
    type      |S4 ...
    sftlf     (lat, lon) float32 dask.array<chunksize=(160, 320), meta=np.ndarray>


In [24]:
# Low-frequency component analysis on loaded data

lat=ds_hist['lat']
lon=ds_hist['lon']

In [27]:
ds_rcp85.member_id

<xarray.DataArray 'member_id' (member_id: 42)>
array(['r10i1p1f1', 'r10i1p2f1', 'r11i1p1f1', 'r11i1p2f1', 'r12i1p1f1',
       'r12i1p2f1', 'r13i1p2f1', 'r14i1p1f1', 'r15i1p1f1', 'r16i1p2f1',
       'r17i1p1f1', 'r17i1p2f1', 'r18i1p1f1', 'r18i1p2f1', 'r19i1p1f1',
       'r19i1p2f1', 'r1i1p1f1', 'r1i1p2f1', 'r20i1p1f1', 'r20i1p2f1',
       'r21i1p1f1', 'r21i1p2f1', 'r22i1p2f1', 'r23i1p1f1', 'r23i1p2f1',
       'r24i1p1f1', 'r24i1p2f1', 'r25i1p1f1', 'r25i1p2f1', 'r2i1p2f1',
       'r3i1p1f1', 'r3i1p2f1', 'r4i1p1f1', 'r4i1p2f1', 'r5i1p1f1', 'r5i1p2f1',
       'r6i1p1f1', 'r6i1p2f1', 'r7i1p2f1', 'r8i1p1f1', 'r9i1p1f1', 'r9i1p2f1'],
      dtype=object)
Coordinates:
  * member_id  (member_id) object 'r10i1p1f1' 'r10i1p2f1' ... 'r9i1p2f1'